In [21]:
import torch
import numpy as np
import cvxpy as cp
from constants import linear_probe_dataset_path, linear_probe_weights
from datasets import load_from_disk

LAYER = 33


In [ ]:
X_SPZ = np.array(new_ds["latents"][:])
np.all(X_SPZ >= 0)
X_SZ = X_SPZ.mean(axis=1)
y_S = np.array(new_ds["label"][:])
print(X_SZ.shape, y_S.shape)

In [ ]:
# new_ds = load_from_disk("test.hf")
new_ds = load_from_disk(linear_probe_dataset_path("gb1", LAYER, "sae"))

In [ ]:
ones_S = np.ones(X_SZ.shape[0])
ones_SZ = ones_S[:, None]
X_SZ = np.concatenate([X_SZ, ones_SZ], axis=1)

In [ ]:
# Split the dataset
n = X_SZ.shape[0]
perm = np.random.permutation(n)

idx_train = [s == "train" for s in new_ds["stage"][:]]
idx_train = []
for s in new_ds["stage"][:]:
    idx_train.append(s == "train")
idx_val = []
for s in new_ds["stage"][:]:
    idx_val.append(s == "valid")
idx_test = []
for s in new_ds["stage"][:]:
    idx_test.append(s == "test")

Xtr, ytr = X_SZ[idx_train], y_S[idx_train]
Xva, yva = X_SZ[idx_val], y_S[idx_val]
Xte, yte = X_SZ[idx_test], y_S[idx_test]

In [ ]:
# Define CVXPY problem
d = X_SZ.shape[1]

W = cp.Variable(d, nonneg=True)
lam = cp.Parameter(nonneg=True)

objective = cp.Minimize(
    cp.sum_squares(Xtr @ W - ytr) + lam * cp.norm1(W)
)

prob = cp.Problem(objective)


In [ ]:
from tqdm import tqdm
# Validation Loop
lams = [0.0, 1e-4, 1e-3, 1e-2, 1e-1]
best_lam, best_val = None, np.inf

for l in tqdm(lams):
    lam.value = l
    # prob.solve(warm_start=True, verbose=True)
    prob.solve(warm_start=True)

    yhat_val = Xva @ W.value
    val_err = np.mean((yhat_val - yva)**2)
    print(val_err)

    if val_err < best_val:
        best_val = val_err
        best_lam = l
print(best_lam)

  0%|          | 0/5 [00:00<?, ?it/s]/data/anaconda3/envs/barsaelona_ishan/lib/python3.11/site-packages/cvxpy/problems/problem.py:1539: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
 20%|██        | 1/5 [00:05<00:23,  5.75s/it]

0.6238360448807307


 40%|████      | 2/5 [00:11<00:17,  5.77s/it]

0.5197077507942875


 60%|██████    | 3/5 [00:17<00:11,  5.77s/it]

0.5194331722426712


 80%|████████  | 4/5 [00:23<00:05,  5.75s/it]

0.4843752110192257


100%|██████████| 5/5 [00:28<00:00,  5.74s/it]

0.6554888941590336
0.01


In [ ]:
Xtv = np.vstack([Xtr, Xva])
ytv = np.concatenate([ytr, yva])

W = cp.Variable(d, nonneg=True)
lam = cp.Parameter(nonneg=True)

objective = cp.Minimize(
    cp.sum_squares(Xtv @ W - ytv) + lam * cp.norm1(W)
)

prob = cp.Problem(objective)

lam.value = best_lam
prob.solve(verbose=True)

W_star = W.value

(CVXPY) Jan 03 10:16:14 PM: Your problem has 4097 variables, 0 constraints, and 1 parameters.
(CVXPY) Jan 03 10:16:14 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 03 10:16:14 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 03 10:16:14 PM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Jan 03 10:16:14 PM: Compiling problem (target solver=OSQP).
(CVXPY) Jan 03 10:16:14 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> OSQP
(CVXPY) Jan 03 10:16:14 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jan 03 10:16:14 PM: Applying reduction Qp2SymbolicQp
(CVXPY) Jan 03 10:16:14 PM: Applying reduction QpMatrixStuffing
(CVXPY) Jan 03 10:16:14 PM: Applying reduction OSQP
(CVXPY) Jan 03 10:16:14 PM: Finished problem compilation (took 2.864e-02 seconds).
(CVXPY) Jan 03 10:16:14 PM: (Subsequent compilations of this problem, using the same arguments, should take

                                     CVXPY                                     
                                     v1.7.5                                    
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
-----------------------------------------------------------------
           OSQP v1.0.0  -  Operator Splitting QP Solver
              (c) The OSQP Developer Team
-----------------------------------------------------------------
problem:  variables n = 8288, constraints m = 12385
          nnz(P) + nnz(A) = 158426
settings: algebra = Built-in,
          OSQ

(CVXPY) Jan 03 10:16:20 PM: Problem status: user_limit
(CVXPY) Jan 03 10:16:20 PM: Optimal value: 5.926e+02
(CVXPY) Jan 03 10:16:20 PM: Compilation took 2.864e-02 seconds
(CVXPY) Jan 03 10:16:20 PM: Solver (including time spent in interface) took 6.251e+00 seconds


10000   6.2578e+00   2.32e-02   1.65e-05  -2.53e+00   2.32e-02   5.09e-01    6.25e+00s

status:               maximum iterations reached
number of iterations: 10000
run time:             6.25e+00s
optimal rho estimate: 9.34e-01

-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------


In [ ]:
Xtv = np.vstack([Xtr, Xva])
ytv = np.concatenate([ytr, yva])

W = cp.Variable(d, nonneg=True)
lam = cp.Parameter(nonneg=True)

objective = cp.Minimize(
    cp.sum_squares(Xtv @ W - ytv) + lam * cp.norm1(W)
)

prob = cp.Problem(objective)

lam.value = best_lam
prob.solve()

W_star = torch.tensor(W.value)
model_path = linear_probe_weights("gb1", LAYER, "sae")
torch.save(W_star, model_path)
print("Saved to: ", model_path)
print("Weight vector shape: ", W_star.shape)

Saved to:  /data/ishan/barSAElona_GUIDEola/gb1_l33_sae_act.pt
Weight vector shape:  torch.Size([4097])
